<a href="https://colab.research.google.com/github/FaridRash/IGNIS/blob/main/Notebooks/SNR/Calculation/SNR_Calculation_NEDT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

"""
Since we want to study the temperature difference over the fumarole and not to
detect the fumarole, instead of calculating the SNR as a difference between the
radiation of the fumarola - the radiation from the background in the same pixel
and normalise that, we could consider the difference in intensity of radiation
between the pixel containing the fumarola and the pixel near it. For this I mean,
we consider the total radiation from the pixel of the fumarole (so the radiation
of the fumarole + radiation of the background), than we consider the radiation
of a pixel nearby that only has the background. We can divide the difference
of these two and divide it by the signal we would receive by the NEDT detectable
by the camera
"""

'\nSince we want to study the temperature difference over the fumarole and not to\ndetect the fumarole, instead of calculating the SNR as a difference between the\nradiation of the fumarola - the radiation from the background in the same pixel\nand normalise that, we could consider the difference in intensity of radiation\nbetween the pixel containing the fumarola and the pixel near it. For this I mean,\nwe consider the total radiation from the pixel of the fumarole (so the radiation\nof the fumarole + radiation of the background), than we consider the radiation\nof a pixel nearby that only has the background. We can divide the difference\nof these two and divide it by the signal we would receive by the NEDT detectable\nby\xa0the\xa0camera\n'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

#Constants

In [3]:
c1 = 3.7418e8
c2 = 1.4388e4
h = 6.626e-34
c = 3e8

In [4]:
epsilon = 0.95
tau_0 = 0.89
eta = 0.7
t = 0.01
focal_length = 72.8
F_number = 1.05
D = (focal_length / F_number) / 1000
NEDT = 0.0441 # NEDT = 44.1mK
lambda_range = np.arange(8, 14.1, 0.1)
delta_lambda = 0.1

#Variables

In [5]:
Tb = 333.15
R = 460000
theta = 20
tau_a = 0.5
Tt = 433.15
At = 100

#Functions

In [6]:
def compute_sT(T):
    M_lambda = c1 / (lambda_range**5 * (np.exp(c2 / (lambda_range * T)) - 1))
    E_lambda = epsilon * M_lambda
    S_lambda = lambda_range * E_lambda * tau_a * tau_0 * eta * delta_lambda
    S_sum = np.sum(S_lambda)
    print(f'T: {T}')
    print(f'S_sum: {S_sum}')
    return S_sum

In [7]:
def compute_s(theta_deg, R, A0, S_sum):
  print(f'theta_deg: {theta_deg}')
  print(f'R: {R}')
  print(f'A0: {A0}')
  print(f'S_sum: {S_sum}')
  cos_theta = np.cos(np.deg2rad(theta_deg))
  S_total = (D**2 * t * cos_theta * A0) / (4 * R**2 * h * c) * S_sum
  return S_total

In [8]:
def GSD(h):
  return (h * 0.000012 / 0.0728) ** 2

#Sb

In [9]:
Ab = GSD(R)
s_Tb= compute_sT(Tb)
print(f'S_Tb: {s_Tb:.2f}')

T: 333.15
S_sum: 883.0481340325063
S_Tb: 883.05


In [10]:
s_b = compute_s(theta, R, Ab, s_Tb)
print(f'Sb: {s_b:.2f}')

theta_deg: 20
R: 460000
A0: 5749.305639415529
S_sum: 883.0481340325063
Sb: 1363080202047365.25


#SNEDT

In [11]:
print(f'Tb: {Tb}')
Tnedt = Tb + NEDT
s_Tnedt = compute_sT(Tnedt)
s_N = s_Tnedt - s_Tb
print(f's_NEDT: {s_N:.2f}')
s_NEDT = compute_s(theta, R, Ab, s_N)
print(f'S_NEDT: {s_NEDT:.2f}')


Tb: 333.15
T: 333.1941
S_sum: 883.5269996696619
s_NEDT: 0.48
theta_deg: 20
R: 460000
A0: 5749.305639415529
S_sum: 0.4788656371556499
S_NEDT: 739180849029.05


#Smix

In [12]:
s_Tt= compute_sT(Tt)
print(f'St: {s_Tt:.2f}')

T: 433.15
S_sum: 2322.8442263205716
St: 2322.84


In [13]:
s_t = compute_s(theta, R, At, s_Tt)
print(f'St: {s_t:.2f}')

theta_deg: 20
R: 460000
A0: 100
S_sum: 2322.8442263205716
St: 62365114246135.16


In [14]:
s_mix = (s_b *(1-(At/Ab))) + s_t
print(f'Smix: {s_mix:.2f}')

Smix: 1401736710636802.00


In [15]:
print(f's_b: {s_b}')
print(f's_t: {s_t}')
print(f's_mix: {s_mix}')
print(f's_NEDT: {s_NEDT}')

s_b: 1363080202047365.2
s_t: 62365114246135.164
s_mix: 1401736710636802.0
s_NEDT: 739180849029.0469


#SNR

In [16]:
SNR = (s_mix - s_b) / s_NEDT
print(f'Sb: {SNR:.2f}')

Sb: 52.30
